In [1]:
!pip install -q mediapipe

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
downstream 1.15.2 requires numpy>=2.0.0, but you have numpy 1.26.4 which is incompatible.
tensorflow-intel 2.18.0 requires ml-dtypes<0.5.0,>=0.4.0, but you have ml-dtypes 0.5.1 which is incompatible.


In [2]:
import joblib

# Load trained model
model = joblib.load("mlp_expression_model.pkl")

# Load LabelEncoder to decode predictions (optional)
import pickle
le = pickle.load(open("label_encoder.pkl", "rb"))  # If you've saved it (see next block)


In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# MediaPipe setup
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1)

# OpenCV webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to RGB
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process frame
    results = face_mesh.process(rgb)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Extract 468 landmarks
            landmarks = face_landmarks.landmark
            x_vals = [lm.x for lm in landmarks]
            y_vals = [lm.y for lm in landmarks]
            z_vals = [lm.z for lm in landmarks]

            # Combine and reshape into feature vector
            features = np.array(x_vals + y_vals + z_vals).reshape(1, -1)

            # Predict
            pred = model.predict(features)
            label = le.inverse_transform(pred)[0]

            # Show prediction
            cv2.putText(frame, f'Expression: {label}', (30, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display
    cv2.imshow("Expression Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


c:\Users\tarek\anaconda3\envs\tf_gpu\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MLPClassifier was fitted with feature names
  warnings.warn(


ValueError: X has 1404 features, but MLPClassifier is expecting 1434 features as input.

: 